<a href="https://colab.research.google.com/github/sahug/ds-tensorflow-colab/blob/master/Tensorflow%20-%20Text%20Classification%20using%20Tensorflow%20Hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tensorflow - Text Classification using Tensorflow Hub**

The tutorial demonstrates the basic application of transfer learning with TensorFlow Hub and Keras.

In transfer learning we use the knowledge from the previous model to build next model. In here we will use embedding from hub to build a Text Classification Model.

In [1]:
!pip install tensorflow-hub
!pip install tensorflow-datasets

In [2]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [3]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.8.0
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


**Download and Exlpore Data**

In [5]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=("train[:60%]", "train[60%:]", "test"),
    as_supervised=True
)

In [7]:
train_example_batch, train_labels_batch =  next(iter(train_data.batch(10)))
train_example_batch, train_labels_batch

(<tf.Tensor: shape=(10,), dtype=string, numpy=
 array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
        b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fe

**Build the Model**

The neural network is created by stacking layers—this requires three main architectural decisions:

- How to represent the text?
- How many layers to use in the model?
- How many hidden units to use for each layer?

Let's first create a **Keras** layer that uses a **TensorFlow Hub model** to embed the sentences, and try it out on a couple of input examples. Note that no matter the length of the input text, the output shape of the embeddings is: **(num_examples, embedding_dimension)**.

**Embedding**

In [8]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_example_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

**Model**

In [9]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation="relu"))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


**Loss Function and Optimizer**

In [10]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

**Train**

In [11]:
history = model.fit(train_data.shuffle(10000).batch(512), epochs=10, validation_data=validation_data.batch(512), verbose=1)

Epoch 1/10
30/30 [==============================] - 20s 615ms/step - loss: 1.9517 - accuracy: 0.5561 - val_loss: 0.8196 - val_accuracy: 0.5819
Epoch 2/10
30/30 [==============================] - 19s 618ms/step - loss: 0.6798 - accuracy: 0.6615 - val_loss: 0.6617 - val_accuracy: 0.6566
Epoch 3/10
30/30 [==============================] - 19s 610ms/step - loss: 0.5468 - accuracy: 0.7394 - val_loss: 0.5992 - val_accuracy: 0.7170
Epoch 4/10
30/30 [==============================] - 19s 607ms/step - loss: 0.4538 - accuracy: 0.8059 - val_loss: 0.5520 - val_accuracy: 0.7538
Epoch 5/10
30/30 [==============================] - 19s 610ms/step - loss: 0.3753 - accuracy: 0.8523 - val_loss: 0.5195 - val_accuracy: 0.7910
Epoch 6/10
30/30 [==============================] - 19s 605ms/step - loss: 0.3055 - accuracy: 0.8901 - val_loss: 0.4954 - val_accuracy: 0.8128
Epoch 7/10
30/30 [==============================] - 19s 605ms/step - loss: 0.2503 - accuracy: 0.9134 - val_loss: 0.4911 - val_accuracy: 0.8282

**Evaluate**

In [12]:
result = model.evaluate(test_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, result):
  print("%s: %.3f" % (name, value))

49/49 - 5s - loss: 0.5543 - accuracy: 0.8298 - 5s/epoch - 94ms/step
loss: 0.554
accuracy: 0.830
